*Read data form bronze layer*

In [0]:
%sql
SELECT * FROM bara_slaes_project.bronze.erp_px_cat_g1v2 LIMIT 5;

In [0]:
df=spark.table("bara_slaes_project.bronze.erp_px_cat_g1v2")

---
*Data Transformation*

In [0]:
# *Rename Cols*
rename_mapping = {
    'ID':'Category_ID',
    'CAT':'Category_Name',
    'SUBCAT':'Sub_Category_Name',
    'MAINTENANCE':'Maintenance'
}

df = df.withColumnsRenamed(rename_mapping)

In [0]:
## Trim
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
for field in df.schema.fields:
    if isinstance(field.dataType,StringType):
        df=df.withColumn(field.name,trim(col(field.name)))

---
*Write into Silver layer*

In [0]:
df.write.mode("overwrite")\
    .format("delta")\
        .saveAsTable("bara_slaes_project.silver.erp_category_info")

In [0]:
%sql
SELECT * FROM bara_slaes_project.silver.erp_category_info LIMIT 5;

In [0]:
%sql
SELECT Category_ID ,COUNT(*)
 FROM bara_slaes_project.silver.erp_category_info
 GROUP BY Category_ID HAVING COUNT(*) > 1